In [7]:
import numpy as np
import os
import sys
import math
import h5py
import readsnap

In [8]:
#define particle datatype
vect = np.dtype([('x', np.float32),('y', np.float32),('z', np.float32)])
part = np.dtype([('ID', np.ulonglong),('pos', vect),('vel', vect)])

In [9]:
snap = 117
snapshot_fname = '/cosma7/data/dp004/dc-smit4/Daemmerung/Planck2013-Npart_512_Box_750-Fiducial/snapdir_{0:03d}/Planck2013-L750-N512-Fiducial_{0:03d}.0'.format(snap)
#snapshot_fname =  '/cosma6/data/dp004/dc-smit4/Daemmerung/Planck2013-Npart_2048_Box_3000-Fiducial/run1/snapdir_{0:03d}/Planck2013-L3000-N2048-Fiducial_{0:03d}.0'.format(snap)
 
head = readsnap.snapshot_header(snapshot_fname)

In [31]:
# load existing ordered  snapshot
ordered_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)
with h5py.File(ordered_fname,'a') as f:
    h = f.create_dataset("Header", dtype = 'f')
    h.attrs.create('Time', head.time)
    h.attrs.create('Redshift', head.redshift)
    h.attrs.create('BoxSize', head.boxsize)
    h.attrs.create('NumFilesPerSnapshot', 1)
    h.attrs.create('Omega0', head.omega_m)
    h.attrs.create('OmegaLambda', head.omega_l)
    h.attrs.create('HubbleParam', head.hubble)
    h.attrs.create('MassTable', head.massarr)
    h.attrs.create('NumPart_ThisFile', head.npart)
    h.attrs.create('NumPart_Total', head.nall + head.nall_hi * 2**32)
    h.attrs.create('Flag_Cooling', head.cooling)
    h.attrs.create('Format', 'hdf5')
    
    ds = f['ordered_parts']
    f.create_dataset('PartType1/Coordinates',data = ds['pos'])
    f.create_dataset('PartType1/Velocities',data = ds['vel'])
    f.create_dataset('PartType1/ParticleIDs',data =ds['ID'])
    
    print('New ds created')
    del f['ordered_parts']

New ds created


In [21]:
def update_header(ordered_fname):
     with h5py.File(ordered_fname,'a') as fi:
        h = fi['Header']
        h.attrs['NumPart_ThisFile'] = h.attrs['NumPart_Total']
        #print (h.attrs['NumFilesPerSnapshot'], h.attrs['NumPart_Total'], h.attrs['NumPart_ThisFile']  )
        #print (fi.keys())
        print('Snap {0:2d} done'.format(snap))

In [23]:
for snap in range(113, 128):
    # Check that file actually exists for this snapshot
    #ordered_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)
    ordered_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)

    if os.path.exists(ordered_fname):
        update_header(ordered_fname)
    else:
        print('Snap {0:2d} missing'.format(snap))

Snap 113 done
Snap 114 done
Snap 115 done
Snap 116 done
Snap 117 done
Snap 118 missing
Snap 119 done
Snap 120 done
Snap 121 done
Snap 122 missing
Snap 123 done
Snap 124 done
Snap 125 missing
Snap 126 done
Snap 127 done


In [19]:
snap = 117
ordered_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)
with h5py.File(ordered_fname,'r') as f:
    print(f.keys())
    h = f['Header']
    print (h.attrs['NumFilesPerSnapshot'], h.attrs['NumPart_Total'], h.attrs['NumPart_ThisFile']  )
    pts = f['PartType1']
    print(pts[:10:]['ID'])
    print(pts[:10:]['pos'])
    print(pts[:10:]['vel'])

    parts = pts[:10:]

<KeysViewHDF5 ['Header', 'PartType1']>
1 [        0 134217728         0         0         0         0] [     0 801936      0      0      0      0]
[0 1 2 3 4 5 6 7 8 9]
[(  0.       ,   0.     ,  0.       ) (749.62305  , 742.13635,  1.3334603)
 (749.4407   , 741.81793,  1.4683721) (  0.9115747, 742.3372 ,  3.3210018)
 (  1.7837795, 742.4564 ,  5.2505813) (  1.587592 , 742.79974,  7.188479 )
 (  1.3806626, 743.377  ,  9.275988 ) (  1.3175293, 743.94946, 10.408505 )
 (  1.0517321, 744.7907 , 12.741033 ) (  1.304157 , 745.4803 , 14.204021 )]
[(  0.      ,    0.     ,   0.      ) ( 60.968998, -386.59592,  61.999577)
 (-11.403959, -401.2491 ,  38.813835) ( 33.84523 , -379.94733,  23.415049)
 ( 75.8855  , -377.05493,  48.98939 ) ( 74.35765 , -361.78244,  64.71091 )
 ( 73.2155  , -332.175  ,  90.250916) ( 72.5216  , -311.97   ,  81.0137  )
 ( 61.45097 , -272.02625, 123.73509 ) ( 66.048645, -237.63681, 133.8675  )]


In [6]:
parts

array([(0, (  0.       ,   0.     ,  0.       ), (  0.      ,    0.     ,   0.      )),
       (1, (749.62305  , 742.13635,  1.3334603), ( 60.968998, -386.59592,  61.999577)),
       (2, (749.4407   , 741.81793,  1.4683721), (-11.403959, -401.2491 ,  38.813835)),
       (3, (  0.9115747, 742.3372 ,  3.3210018), ( 33.84523 , -379.94733,  23.415049)),
       (4, (  1.7837795, 742.4564 ,  5.2505813), ( 75.8855  , -377.05493,  48.98939 )),
       (5, (  1.587592 , 742.79974,  7.188479 ), ( 74.35765 , -361.78244,  64.71091 )),
       (6, (  1.3806626, 743.377  ,  9.275988 ), ( 73.2155  , -332.175  ,  90.250916)),
       (7, (  1.3175293, 743.94946, 10.408505 ), ( 72.5216  , -311.97   ,  81.0137  )),
       (8, (  1.0517321, 744.7907 , 12.741033 ), ( 61.45097 , -272.02625, 123.73509 )),
       (9, (  1.304157 , 745.4803 , 14.204021 ), ( 66.048645, -237.63681, 133.8675  ))],
      dtype=[('ID', '<u8'), ('pos', [('x', '<f4'), ('y', '<f4'), ('z', '<f4')]), ('vel', [('x', '<f4'), ('y', '<f4'), (

In [51]:
snap = 70
new_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)
with h5py.File(new_fname,'w') as f:
        f.create_dataset('PartType1',data = parts)

array([(0, (  0.       ,   0.     ,  0.       ), (  0.      ,    0.     ,   0.      )),
       (1, (749.62305  , 742.13635,  1.3334603), ( 60.968998, -386.59592,  61.999577)),
       (2, (749.4407   , 741.81793,  1.4683721), (-11.403959, -401.2491 ,  38.813835)),
       (3, (  0.9115747, 742.3372 ,  3.3210018), ( 33.84523 , -379.94733,  23.415049)),
       (4, (  1.7837795, 742.4564 ,  5.2505813), ( 75.8855  , -377.05493,  48.98939 )),
       (5, (  1.587592 , 742.79974,  7.188479 ), ( 74.35765 , -361.78244,  64.71091 )),
       (6, (  1.3806626, 743.377  ,  9.275988 ), ( 73.2155  , -332.175  ,  90.250916)),
       (7, (  1.3175293, 743.94946, 10.408505 ), ( 72.5216  , -311.97   ,  81.0137  )),
       (8, (  1.0517321, 744.7907 , 12.741033 ), ( 61.45097 , -272.02625, 123.73509 )),
       (9, (  1.304157 , 745.4803 , 14.204021 ), ( 66.048645, -237.63681, 133.8675  ))],
      dtype=[('ID', '<u8'), ('pos', [('x', '<f4'), ('y', '<f4'), ('z', '<f4')]), ('vel', [('x', '<f4'), ('y', '<f4'), (

In [49]:
134217728 * 32

4294967296

In [ ]:
4294971394